In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import json

In [2]:
data_dir = '../data'
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

In [3]:
driver = webdriver.Chrome() 
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)'
}

In [4]:
def read(qna_id):
    url = f'https://www.lawtalk.co.kr/qna/{qna_id}'
    driver.get(url)
    time.sleep(2)

    # 카테고리
    categories = []
    question_keywords_tag = driver.find_element(By.CLASS_NAME, 'question-keywords')
    category_tags = question_keywords_tag.find_elements(By.CLASS_NAME, 'question-keyword')
    for category_tag in category_tags:
        category = category_tag.get_attribute('field')
        categories.append(category)

    # 질문 제목
    question_title_tag = driver.find_element(By.CLASS_NAME, 'question-title')
    question_title = question_title_tag.get_attribute('innerText').strip()

    # 질문 본문
    question_body_tag = driver.find_element(By.CLASS_NAME, 'question-body')
    question_body = question_body_tag.get_attribute('innerText').strip()

    # 추천 키워드 / 해시태그
    recommend_keywords = []
    recommend_keywords_tag = driver.find_element(By.CLASS_NAME, 'recommend-question-keywords')
    recommend_keyword_tags = recommend_keywords_tag.find_elements(By.CLASS_NAME, 'recommend-question-keyword')
    for recommend_keyword_tag in recommend_keyword_tags:
        span = recommend_keyword_tag.find_element(By.TAG_NAME, 'span')
        keyword = span.get_attribute('innerText').strip()
        recommend_keywords.append(keyword)

    # 답변 리스트
    answers = []
    qna_list_feed_tag = driver.find_element(By.CLASS_NAME, 'qna-list-feed')
    qna_list_item_tags = qna_list_feed_tag.find_elements(By.CLASS_NAME, 'qna-list-item')
    for qna_list_item_tag in qna_list_item_tags:
        answer_body_tag = qna_list_item_tag.find_element(By.CLASS_NAME, 'answer-body')
        answer = answer_body_tag.get_attribute('innerText').strip()
        if len(answer) > 50:
            answers.append(answer)

    if answers:
        return {
            'categories': categories,
            'question_title': question_title,
            'question_body': question_body,
            'recommend_keywords': recommend_keywords,
            'answers': answers,
        }

In [5]:
start_id = 441738
if os.path.exists(data_dir):
    qna_ids = sorted([int(f.split('.')[0]) for f in os.listdir(data_dir) if not f.startswith('.')])
    if qna_ids:
        start_id = qna_ids[0]
print(f'start_id: {start_id}')

for qna_id in range(start_id, 0, -1):
    try:
        content = read(qna_id)
    except Exception:
        continue
    if content:
        with open(f'{data_dir}/{qna_id}.json', 'w', encoding='utf-8') as f:
            json.dump(content, f, ensure_ascii=False, indent=4)

start_id: 259597
